In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import r2_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
import random
from collections import OrderedDict
from scipy.stats import pearsonr

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
#os.environ["CUDA_VISIBLE_DEVICES"]="1";
#torch.cuda.set_device(1)

In [7]:
dataset_dir = '/home/matthew/Programming/Kaggle/MolecularProperties/dataset'

In [8]:
def load_dataset():
    set_path = os.path.join(dataset_dir,'train.csv')
    set_df = pd.read_csv(set_path)
    X = set_df.molecule_name.values
    y = set_df.scalar_coupling_constant.values
    return train_test_split(X, y, test_size=0.05)

In [20]:
def construct_multigraph(mol):
    g = OrderedDict({})
    h = OrderedDict({})

    g_path = os.path.join(dataset_dir,'multigraphs/%s_g.pkl' % mol)
    with open(g_path, 'rb') as read_file:
        g_loaded = pickle.load(read_file)
    for key in g_loaded.keys():
        feats = []
        for bond in g_loaded[key]:
            bond = (Variable(torch.FloatTensor(bond[0]).view(1, 5)).to(DEVICE),bond[1])
            feats.append(bond)
        g[key] = feats

    h_path = os.path.join(dataset_dir,'multigraphs/%s_h.pkl' % mol)
    with open(h_path, 'rb') as read_file:
        h_loaded = pickle.load(read_file)
    for key in h_loaded.keys():
        atom = Variable(torch.FloatTensor(h_loaded[key]).view(1, 7)).to(DEVICE)
        h[key] = atom

    return g,h

In [27]:
class MPNN(nn.Module):
    def __init__(self):
        super(MPNN, self).__init__()
        self.R = nn.Linear(14,128)
        self.U0 = nn.Linear(19, 7)
        self.U1 = nn.Linear(19, 7)
        self.U2 = nn.Linear(19, 7)
        self.V0 = nn.Linear(7, 7)
        self.V1 = nn.Linear(7, 7)
        self.V2 = nn.Linear(7, 7)
        self.E = nn.Linear(5, 5)
        self.linear = nn.Linear(128, 1)

    def forward(self, g, h):
        y_hat = Variable(torch.zeros(1, 1)).to(DEVICE)
        h2 = h
        for k in range(0, T):
            for v in g.keys():
                neighbors = g[v]
                for neighbor in neighbors:
                    e_vw = neighbor[0] # feature variable
                    w = neighbor[1]

                    if k == 0:
                        m_w = self.V0(h[w])
                    elif k == 1:
                        m_w = self.V1(h[w])
                    elif k == 2:
                        m_w = self.V2(h[w])
                        
                    m_e_vw = self.E(e_vw)
                    reshaped = torch.cat( (h[v], m_w, m_e_vw), 1)
                    
                    if k == 0:
                        selu = F.selu(self.U0(reshaped))
                    elif k == 1:
                        selu = F.selu(self.U1(reshaped))
                    elif k == 2:
                        selu = F.selu(self.U2(reshaped))
                    
                    h[v] = selu

        catted_reads = map(lambda x: torch.cat([h[x[0]], h2[x[1]]], 1), zip(h2.keys(), h.keys()))
        activated_reads = map(lambda x: F.selu( self.R(x) ), catted_reads)
        readout = Variable(torch.zeros(1, 128)).to(DEVICE)
        for read in activated_reads:
            readout = readout + read
        x = torch.tanh(readout)
        y_hat = self.linear(x)
        return y_hat

In [ ]:
X_train, X_val, y_train, y_val = load_dataset()

T = 3
BATCH_SIZE = 512
N_EPOCH = 10
N_STEPS = int(int(len(X_train) / BATCH_SIZE) * N_EPOCH)
LR = 5e-4
DEVICE = torch.device('cuda:0')

model = MPNN().to(DEVICE)

#model.load_state_dict(torch.load('model_20e.pt'))

y_train = Variable(torch.FloatTensor(y_train), requires_grad=False).to(DEVICE)
y_val = Variable(torch.FloatTensor(y_val), requires_grad=False).to(DEVICE)

epoch_num = 1
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
for i in range(1, N_STEPS):
    model.train()
    optimizer.zero_grad()
    train_loss = Variable(torch.zeros(1, 1)).to(DEVICE)
    y_hats_train = []
    for j in range(0, BATCH_SIZE):
        sample_index = random.randint(0, len(X_train) - 2)
        g, h = construct_multigraph(X_train[sample_index])
        
        y_hat = model(g,h)
        y_hats_train.append(y_hat)
        
        y = y_train[sample_index]
        error = (y_hat - y)**2
        train_loss += error

    train_loss /= Variable(torch.FloatTensor([BATCH_SIZE])).view(1, 1).to(DEVICE)
    train_loss.backward()
    optimizer.step()
    print('step %d/%d' % (i,N_STEPS),end='\r')

    if i % +int(len(X_train) / BATCH_SIZE) == 0: # Run validation prediction
        model.eval()
        val_loss = Variable(torch.zeros(1, 1), requires_grad=False).to(DEVICE)
        y_hats_val = []
        for j in range(0, len(X_val)):
            g, h = construct_multigraph(X_val[j])
            
            y_hat = model(g,h)
            y_hat_cpu = y_hat
            y_hats_val.append(y_hat_cpu.cpu())
            
            y = y_val[j]
            error = (y_hat - y)*(y_hat - y) / Variable(torch.FloatTensor([len(X_val)])).view(1, 1).to(DEVICE)
            val_loss = val_loss + error

        y_hats_val = np.array([x.data.numpy() for x in y_hats_val])
        y_hats_val = y_hats_val.reshape(-1, 1)
        
        y_val_ = y_val
        y_vals = np.array(y_val_.cpu())
        y_vals = y_vals.reshape(-1, 1)

        #print(y_hats_val)

        r2_val_old = r2_score(y_vals, y_hats_val)
        #pearsonr = pearsonr(y_vals, y_hats_val)[0]**2

        train_loss_ = train_loss
        train_loss_ = train_loss.cpu().data.numpy()[0]
        val_loss_ = val_loss
        val_loss_ = val_loss.cpu().data.numpy()[0]
        print('epoch [%i/%i] train_loss [%f] val_loss [%f] r2_val_old [%.4f]' \
                  % (epoch_num, N_EPOCH, train_loss_, val_loss_, r2_val_old))
        epoch_num += 1